## Library Installation

Ensure that the necessary libraries are installed before running the notebook.

In [1]:
# Import necessary libraries
%pip install openpyxl
from openpyxl import load_workbook
import pandas as pd
import os
import re
import itertools

print("Skeleton setup complete!")


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Skeleton setup complete!


## Variable Declaration

Set the variables for file paths, sheet names, and other configurations. Update these variables for each specific project.

In [2]:
# Path to the Excel file (change this for each project)
excel_file_path = '/workspaces/Finetwork-Automation/inbound/Metabase.xlsx'
csv_file_path = '/workspaces/Finetwork-Automation/inbound/Informe de métricas históricas.csv'

# Sheet names for different categories
sheet_active = 'ACTIVOS'
sheet_canceled = 'CANCELADOS'
sheet_pending = 'PTE DE FIRMA'
sheet_value = 'TOP VALOR'
sheet_packs = 'TOP PACKS'
sheet_services = 'TOP SERVICIOS'

# Range to read (change this for each project)
start_row = 8
end_row = 150
usecols = 'A:AD'

print("Variables defined correctly!")

Variables defined correctly!


## Extract Data from pivot canceled sheet

Extract data from pivot canceled sheet and convert it directly to a DataFrame.

In [3]:
import pandas as pd

def load_sheet_as_dataframe(file_path, sheet_name, start_row, usecols):
    """
    Load data from the specified sheet and range into a DataFrame.

    Parameters:
    file_path (str): Path to the Excel file.
    sheet_name (str): Name of the sheet to load data from.
    start_row (int): The row number to start loading data from.
    usecols (str): String indicating the columns to load (e.g., 'A,R').

    Returns:
    pd.DataFrame: The loaded DataFrame.
    """
    # Load data from the specified sheet and range into a DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=usecols, skiprows=start_row-1)
    print(f"Data from '{sheet_name}' sheet loaded successfully.")
    return df

# Parameters for loading the sheet
excel_file_path = '/workspaces/Finetwork-Automation/inbound/Metabase.xlsx'
sheet_name = 'CANCELADOS PIVOT'
start_row = 1  # Start from row 2
usecols = 'A,R'  # Columns A and R

# Extract data from 'CANCELADOS PIVOT' sheet
canceled_pivot_df = load_sheet_as_dataframe(excel_file_path, sheet_name, start_row, usecols)

# Display the DataFrame
print("Canceled Pivot DataFrame:")
display(canceled_pivot_df.head(88))

Data from 'CANCELADOS PIVOT' sheet loaded successfully.
Canceled Pivot DataFrame:


,email_employed,service__status_name
0,manuelvaldes@originaltelecom.es,NaN
1,albaaraujo@originaltelecom.es,CANCELADO
2,albaaraujo@originaltelecom.es,NaN
3,albaaraujo@originaltelecom.es,NaN
4,albaaraujo@originaltelecom.es,NaN
...,...,...
83,laura.eguens@originaltelecom.es,CANCELADO
84,laura.eguens@originaltelecom.es,CANCELADO
85,laura.eguens@originaltelecom.es,CANCELADO
86,laura.eguens@originaltelecom.es,CANCELADO


## Group Canceled values

Blablabla

In [4]:
import pandas as pd

# Agrupar por 'email_employed' y contar el número de veces que aparece 'CANCELADO' en la columna 'service__status_name'
result_df = canceled_pivot_df.groupby('email_employed')['service__status_name'].apply(lambda x: x.eq('CANCELADO').sum()).reset_index()

# Renombrar la columna para que sea más descriptiva
canceled_pivot_df = result_df.rename(columns={'service__status_name': 'Num_Cancelados'})

# Mostrar el DataFrame resultante
print(canceled_pivot_df)

                          email_employed  Num_Cancelados
0        alba.salgado@originaltelecom.es               2
1          albaaraujo@originaltelecom.es              75
2            alberto.valle@finetwork.com               7
3        albertocanto@originaltelecom.es              59
4      albertosanchez@originaltelecom.es              72
..                                   ...             ...
72            tamara.garcia@finetwork.es               0
73       victor.justamante@finetwork.com               0
74  yenniferandreina.perozopetit@cgi.com               3
75     yicel.patricia@originaltelecom.es              47
76     yzabelly.gomes@originaltelecom.es              44

[77 rows x 2 columns]


## Load Agents List

Load the list of all agents from the "Agents" sheet.

In [5]:
# Load the list of agents
agents_df = pd.read_excel(excel_file_path, sheet_name='AGENTES', usecols='A')
agents_list = agents_df.iloc[:, 0].tolist()
print("Agents list loaded successfully!")
print(agents_list)

Agents list loaded successfully!
['albaaraujo@originaltelecom.es', 'albertocanto@originaltelecom.es', 'albertosanchez@originaltelecom.es', 'anasanchez@originaltelecom.es', 'antonio.reina@originaltelecom.es', 'azahara.garcia@originaltelecom.es', 'beatriz.gomez@originaltelecom.es', 'maricarmen.cornejo@originaltelecom.es', 'carolinafuentes@originaltelecom.es', 'cesar.arnaldo@originaltelecom.es', 'david.molero@originaltelecom.es', 'elenaborrero@originaltelecom.es', 'estefania.panea@originaltelecom.es', 'francisco.perdomo@originaltelecom.es', 'gonzalofalcon@originaltelecom.es', 'guillermo.hurtado@originaltelecom.es', 'irati.izaguirre@originaltelecom.es', 'ivan.barroso@originaltelecom.es', 'laura.eguens@originaltelecom.es', 'lailasetati@originaltelecom.es', 'leonor.lopez@originaltelecom.es', 'dolores.cortes@originaltelecom.es', 'manuelvaldes@originaltelecom.es', 'manuelventura@originaltelecom.es', 'mar.aguila@originaltelecom.es', 'mariangeles.bueso@originaltelecom.es', 'mariaarroyo@originalt

## Verify and Complete Data

Verify that all agents are present in each DataFrame. If an agent is missing, add a row with zeros for that agent.

In [6]:
def ensure_all_agents(df, agents_list):
    """
    Ensure all agents are present in the DataFrame. Add missing agents with zero values and remove agents not in the list.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to check and update.
    agents_list (list): The list of all agents.
    
    Returns:
    pd.DataFrame: The updated DataFrame with all agents.
    """
    # Get the list of agents in the DataFrame
    existing_agents = df.iloc[:, 0].tolist()
    
    # Find missing agents
    missing_agents = [agent for agent in agents_list if agent not in existing_agents]
    
    # Add rows for missing agents with zero values
    for agent in missing_agents:
        zero_row = pd.DataFrame([[agent] + [0] * (df.shape[1] - 1)], columns=df.columns)
        df = pd.concat([df, zero_row], ignore_index=True)
    
    # Remove agents not in the agents list
    df = df[df.iloc[:, 0].isin(agents_list)]
    
    print(f"Added {len(missing_agents)} missing agents and removed {len(existing_agents) - len(df)} agents not in the list.")
    return df

# Apply the function to each DataFrame
canceled_pivot_df = ensure_all_agents(canceled_pivot_df, agents_list)

# Display the updated DataFrames
print("Canceled Pivot DataFrame after ensuring all agents:")
display(canceled_pivot_df.head(66))

Added 1 missing agents and removed 34 agents not in the list.
Canceled Pivot DataFrame after ensuring all agents:


,email_employed,Num_Cancelados
1,albaaraujo@originaltelecom.es,75
3,albertocanto@originaltelecom.es,59
4,albertosanchez@originaltelecom.es,72
5,anasanchez@originaltelecom.es,67
6,antonio.reina@originaltelecom.es,81
7,azahara.garcia@originaltelecom.es,18
9,beatriz.gomez@originaltelecom.es,76
10,carolinafuentes@originaltelecom.es,79
12,cesar.arnaldo@originaltelecom.es,30
15,david.molero@originaltelecom.es,82


## Assign Values to Emails

Assign numerical values to each email and add them as a new column in the DataFrames.

In [7]:
# Dictionary mapping emails to their respective values
email_values = {
    'albaaraujo@originaltelecom.es': 1,
    'albertocanto@originaltelecom.es': 2,
    'albertosanchez@originaltelecom.es': 3,
    'anasanchez@originaltelecom.es': 4,
    'antonio.reina@originaltelecom.es': 5,
    'azahara.garcia@originaltelecom.es': 6,
    'beatriz.gomez@originaltelecom.es': 7,
    'maricarmen.cornejo@originaltelecom.es': 8,
    'carolinafuentes@originaltelecom.es': 9,
    'cesar.arnaldo@originaltelecom.es': 10,
    'david.molero@originaltelecom.es': 11,
    'elenaborrero@originaltelecom.es': 12,
    'estefania.panea@originaltelecom.es': 13,
    'francisco.perdomo@originaltelecom.es': 14,
    'gonzalofalcon@originaltelecom.es': 15,
    'guillermo.hurtado@originaltelecom.es': 16,
    'irati.izaguirre@originaltelecom.es': 17,
    'ivan.barroso@originaltelecom.es': 18,
    'laura.eguens@originaltelecom.es': 19,
    'lailasetati@originaltelecom.es': 20,
    'leonor.lopez@originaltelecom.es': 21,
    'dolores.cortes@originaltelecom.es': 22,
    'manuelvaldes@originaltelecom.es': 23,
    'manuelventura@originaltelecom.es': 24,
    'mar.aguila@originaltelecom.es': 25,
    'mariangeles.bueso@originaltelecom.es': 26,
    'mariaarroyo@originaltelecom.es': 27,
    'maria.torres@originaltelecom.es': 28,
    'marta.dorado@originaltelecom.es': 29,
    'mauricio.pozo@originaltelecom.es': 30,
    'miguel.segura@originaltelecom.es': 31,
    'miriam.rodriguez@originaltelecom.es': 32,
    'mar.marchena@originaltelecom.es': 33,
    'natividad.sanchez@originaltelecom.es': 34,
    'nereacerezo@originaltelecom.es': 35,
    'oscar.rivilla@originaltelecom.es': 36,
    'patricia.rios@originaltelecom.es': 37,
    'paulavilla@originaltelecom.es': 38,
    'pilar.deval@originaltelecom.es': 39,
    'sara.elkhelyfy@originaltelecom.es': 40,
    'sergio.vazquez@originaltelecom.es': 41,
    'yicel.patricia@originaltelecom.es': 42,
    'yzabelly.gomes@originaltelecom.es': 43
}

# Add a new column to each DataFrame with the email values
def add_email_values(df, email_values):
    df['email_value'] = df.iloc[:, 0].map(email_values)
    return df

# Apply the function to each DataFrame
canceled_pivot_df = add_email_values(canceled_pivot_df, email_values)

# Display the updated DataFrames with the new 'email_value' column
print("Active DataFrame with email values:")
display(canceled_pivot_df.head())

Active DataFrame with email values:


,email_employed,Num_Cancelados,email_value
1,albaaraujo@originaltelecom.es,75,1
3,albertocanto@originaltelecom.es,59,2
4,albertosanchez@originaltelecom.es,72,3
5,anasanchez@originaltelecom.es,67,4
6,antonio.reina@originaltelecom.es,81,5


## Sort DataFrames by Email Values

Sort the DataFrames based on the numerical values assigned to the emails.

In [8]:
# Sort each DataFrame by the 'email_value' column
def sort_by_email_value(df):
    df = df.sort_values(by='email_value')
    return df

# Apply the sorting function to each DataFrame
canceled_pivot_df = sort_by_email_value(canceled_pivot_df)

# Display the sorted DataFrames
print("Sorted Active DataFrame:")
display(canceled_pivot_df.head(88)) 

Sorted Active DataFrame:


,email_employed,Num_Cancelados,email_value
1,albaaraujo@originaltelecom.es,75,1
3,albertocanto@originaltelecom.es,59,2
4,albertosanchez@originaltelecom.es,72,3
5,anasanchez@originaltelecom.es,67,4
6,antonio.reina@originaltelecom.es,81,5
7,azahara.garcia@originaltelecom.es,18,6
9,beatriz.gomez@originaltelecom.es,76,7
48,maricarmen.cornejo@originaltelecom.es,44,8
10,carolinafuentes@originaltelecom.es,79,9
12,cesar.arnaldo@originaltelecom.es,30,10


## Remove 'email_value' Column

After sorting the DataFrames based on the email values, the 'email_value' column should be removed to prevent interference with further calculations.

In [9]:
# Function to remove the 'email_value' column
def remove_email_value_column(df):
    if 'email_value' in df.columns:
        df = df.drop(columns=['email_value'])
    return df

# Apply the function to each DataFrame
canceled_pivot_df = remove_email_value_column(canceled_pivot_df)

# Display the updated DataFrames without the 'email_value' column
print("Active DataFrame after removing 'email_value' column:")
display(canceled_pivot_df.head(88))

Active DataFrame after removing 'email_value' column:


,email_employed,Num_Cancelados
1,albaaraujo@originaltelecom.es,75
3,albertocanto@originaltelecom.es,59
4,albertosanchez@originaltelecom.es,72
5,anasanchez@originaltelecom.es,67
6,antonio.reina@originaltelecom.es,81
7,azahara.garcia@originaltelecom.es,18
9,beatriz.gomez@originaltelecom.es,76
48,maricarmen.cornejo@originaltelecom.es,44
10,carolinafuentes@originaltelecom.es,79
12,cesar.arnaldo@originaltelecom.es,30


## Obtain Data from Canceled Pivot DataFrames

Obtain the values to update the Excel File

checks

In [10]:
# Verificar los nombres de las columnas
print("Nombres de las columnas:", canceled_pivot_df.columns.tolist())
print("Primeras filas del DataFrame:")
print(result_df.head())
print("Tipos de datos de las columnas:")
print(result_df.dtypes)
# Quitar espacios de los nombres de las columnas
result_df.columns = result_df.columns.str.strip()

# Verificar nombres de columnas después de limpiar espacios
print("Nombres de las columnas después de limpiar espacios:", result_df.columns.tolist())
# Verificar duplicados en los nombres de las columnas
print("¿Hay nombres de columnas duplicados?:", result_df.columns.duplicated().any())
print("Filas con valores NaN en 'Num_Cancelados':")
# Asegurarse de que estamos accediendo a la columna correcta
print("Valores en la columna 'Num_Cancelados':")
print(result_df['service__status_name'].tolist())

# Alternativamente, si sospechamos que el nombre tiene algún error
print("Valores en la columna que parece ser 'Num_Cancelados':")
for col in result_df.columns:
    print(f"Probando columna: {col}")
    print(result_df[col].tolist()[:5])








Nombres de las columnas: ['email_employed', 'Num_Cancelados']
Primeras filas del DataFrame:
                      email_employed  service__status_name
0    alba.salgado@originaltelecom.es                     2
1      albaaraujo@originaltelecom.es                    75
2        alberto.valle@finetwork.com                     7
3    albertocanto@originaltelecom.es                    59
4  albertosanchez@originaltelecom.es                    72
Tipos de datos de las columnas:
email_employed          object
service__status_name     int64
dtype: object
Nombres de las columnas después de limpiar espacios: ['email_employed', 'service__status_name']
¿Hay nombres de columnas duplicados?: False
Filas con valores NaN en 'Num_Cancelados':
Valores en la columna 'Num_Cancelados':
[2, 75, 7, 59, 72, 67, 81, 18, 8, 76, 79, 1, 30, 0, 0, 82, 2, 54, 54, 67, 1, 30, 8, 29, 13, 19, 1, 4, 71, 1, 1, 80, 69, 30, 24, 2, 1, 40, 96, 53, 2, 63, 1, 43, 21, 60, 1, 48, 44, 2, 30, 44, 92, 1, 58, 8, 51, 0, 52, 0, 4, 37

In [11]:
# Obtener los valores de la columna 'Num_Cancelados'
canceled_pivot_values = canceled_pivot_df['Num_Cancelados'].values

# Mostrar los valores obtenidos
print("Valores calculados para 'Canceled Pivot' DataFrame:")
print(canceled_pivot_values[:66])  # Mostrar solo los primeros 5 para verificar

Valores calculados para 'Canceled Pivot' DataFrame:
[75 59 72 67 81 18 76 44 79 30 82 67 30 71 80 69 30 24 96 40 53 54 63  0
  1 48 60 21 30 44 92 58 43 51 52 37 59 34 59 45 45 47 44]


### Update finalFile Excel Sheet with Canceled Pivot Data

Update the "finalFile" Excel sheet with the calculated values from the Canceled Pivot DataFrame in column K.

In [12]:
final_file_path ='/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx'
sheet_name = 'GLOBAL AGOSTO'# Change this to the correct sheet name

def update_final_file_canceled(file_path, sheet_name, values):
    """
    Update the specified Excel sheet with the provided values starting from row 3 in column K.
    
    Parameters:
    file_path (str): Path to the Excel file.
    sheet_name (str): Name of the sheet to update.
    values (list or array-like): Values to insert into the sheet.
    """
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column K
    start_row = 3
    column = 11  # Column K
    
    for idx, value in enumerate(values, start=start_row):
        sheet.cell(row=idx, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Canceled data.")

# Usar el código para actualizar el archivo Excel
update_final_file_canceled(final_file_path, sheet_name, canceled_pivot_values)


Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Canceled data.
